In [ ]:
%pip install IMDbPY
%pip install torchnlp
%pip install pytorch-nlp
%pip install pytorch_pretrained_bert

     |████████████████████████████████| 296kB 5.2MB/s 
     |████████████████████████████████| 51kB 2.1MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 5.4MB 15.9MB/s 
     |████████████████████████████████| 591.8MB 27kB/s 
ERROR: torchvision 0.6.1+cu101 has requirement torch==1.5.1, but you'll have torch 1.0.0 which is incompatible.
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.28.1 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstal

     |████████████████████████████████| 92kB 2.9MB/s 
     |████████████████████████████████| 133kB 5.0MB/s 


In [ ]:
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from torchnlp.datasets import imdb_dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [ ]:
from google.colab import files
data_to_load = files.upload()

Saving train.csv to train.csv


In [ ]:
import io
#train = pd.read_csv(io.StringIO(data_to_load["train.csv"]))
train = pd.read_csv("train.csv")
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
train['total'] = train['text']+train['title']+train['author']

In [ ]:
clean_train_set = train.dropna()
len(clean_train_set)

18285

In [ ]:
train_set ,test_set  = train_test_split(clean_train_set, test_size = 0.33 , random_state = 42)
crop_train_set = train_set[:3000]
crop_test_set = test_set[:700]
len(crop_train_set)
crop_train_set.head()

,id,title,author,text,label,total
14795,14795,I Envy You,Israel Shamir,Here's something interesting from The Unz Revi...,1,Here's something interesting from The Unz Revi...
2526,2526,: Information,Robert Parry,Trumps Slim Chance for Greatness\nSpecial Re...,1,Trumps Slim Chance for Greatness\nSpecial Re...
17048,17048,Lawsuit Aims to Hold 2 Contractors Accountable...,Sheri Fink and James Risen,Nearly 15 years after the United States adopte...,0,Nearly 15 years after the United States adopte...
11117,11117,Mark Levin: Left Has Turned Violence Mainstrea...,Dr. Susan Berry,CRTV and radio host Mark Levin said Wednesday ...,0,CRTV and radio host Mark Levin said Wednesday ...
12777,12777,GABRIEL: The War on Girls - Breitbart,Brigitte Gabriel,“Do you know you are going to die tonight? ”[T...,0,“Do you know you are going to die tonight? ”[T...


In [ ]:
from collections import Counter
print(Counter(crop_train_set['label'].values))

Counter({0: 1698, 1: 1302})


In [ ]:
crop_train_set1 = crop_train_set[['total','label']].reset_index().drop('index', axis = 1)
crop_test_set1 = crop_test_set[['total','label']].reset_index().drop('index', axis = 1)
crop_train_set1

,total,label
0,Here's something interesting from The Unz Revi...,1
1,Trumps Slim Chance for Greatness\nSpecial Re...,1
2,Nearly 15 years after the United States adopte...,0
3,CRTV and radio host Mark Levin said Wednesday ...,0
4,“Do you know you are going to die tonight? ”[T...,0
...,...,...
2995,The rift in the Republican Party grew deeper o...,0
2996,Share on Facebook Share on Twitter Over 25 mil...,1
2997,(Want to get this briefing by email? Here’s th...,0
2998,November 18: Daily Contrarian Reads By David S...,1


In [ ]:
from dict import dict
dict_train = crop_train_set1.T.to_dict().values()
dict_test = crop_test_set1.T.to_dict().values()
dict_test

dict_values([{'total': ' Russian Spies and America\x92s Reality TV Election By Finian Cunningham\nNovember 14, " Sputnik " - After months of mudslinging and vitriol, US President-elect Donald Trump was greeted this week by a chummy President Obama with a cozy fireside seat at the White House.\nSome media outlets sniped at Trump\x92s surprise election calling him the \x93first reality TV star to become president\x94. In truth, the whole US political system seems to be a reality TV show.\nIn the weeks up to polling day on November 8, Democrat candidate Hillary Clinton and large sections of the US media were denouncing Trump as a \x93Russian stooge\x94. Trump\x92s alleged friendliness towards Russian President Vladimir Putin was said to be proof that the Republican candidate was a \x93Kremlin agent\x94. The Kremlin rejected the accusations as fatuous, pointing out that there was not a scintilla of evidence.\nThe claims, however, were not merely electioneering rhetoric to smear Trump. The 

In [ ]:
pip install dict 

  Created wheel for dict: filename=dict-0.0.0-cp36-none-any.whl size=1643 sha256=837fa8e1b6ef1247761889debac1d9b4a79353045738f668de3c47129ee67b6a
  Stored in directory: /root/.cache/pip/wheels/f5/01/88/7a05efefa96f9f3702ccea679735857a9bbf95ecc305f0a2c0
  Created wheel for values: filename=values-0.0.0-cp36-none-any.whl size=1362 sha256=b40d3207a4e93b3d96fe925400113fcca3444f70b2016617127b04de43bdccb3
  Stored in directory: /root/.cache/pip/wheels/8f/bc/fc/5150d9ef5d54d77e96b02385ad4494babe3b2517cb97b84169
Successfully built dict values


In [ ]:
train_texts, train_labels = list(zip(*map(lambda d: (d['total'], d['label']), dict_train)))
test_texts, test_labels = list(zip(*map(lambda d: (d['total'], d['label']), dict_test)))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))

100%|██████████| 231508/231508 [00:00<00:00, 900870.45B/s]


In [ ]:
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))

In [ ]:
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

In [ ]:
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")

In [ ]:
train_y = np.array(train_labels) == 1
test_y = np.array(test_labels) == 1
train_labels

(1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,


In [ ]:
bert = BertModel.from_pretrained('bert-base-uncased')
x = torch.tensor(train_tokens_ids[:3])
y, pooled = bert(x, output_all_encoded_layers=False)
print('x shape:', x.shape)
print('y shape:', y.shape)
print('pooled shape:', pooled.shape)
# OUTPUT
#x shape :(3, 512)
#y shape: (3, 512, 768)
#pooled shape: (3, 768)

100%|██████████| 407873900/407873900 [00:11<00:00, 36897667.20B/s]


x shape: torch.Size([3, 512])
y shape: torch.Size([3, 512, 768])
pooled shape: torch.Size([3, 768])


In [ ]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens):
        _, pooled_output = self.bert(tokens, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [ ]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)


In [ ]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
test_tokens_tensor = torch.tensor(test_tokens_ids)

In [ ]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(np.array(train_labels).reshape(-1,1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(np.array(test_labels).reshape(-1,1)).float()
train_y_tensor

tensor([[1.],
        [1.],
        [0.],
        ...,
        [0.],
        [1.],
        [0.]])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_dataset = TensorDataset(train_tokens_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler)


test_dataset = TensorDataset(test_tokens_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler)

In [ ]:
EPOCHS = 1
bert_clf = BertBinaryClassifier()
bert_clf.train()

BertBinaryClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertI

In [ ]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):
        token_ids, labels = tuple(t for t in batch_data)
        logits = bert_clf(token_ids)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
        
print(classification_report(test_y, bert_predicted))

              precision    recall  f1-score   support

       False       0.66      0.85      0.74       404
        True       0.66      0.41      0.50       296

   micro avg       0.66      0.66      0.66       700
   macro avg       0.66      0.63      0.62       700
weighted avg       0.66      0.66      0.64       700



In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(test_y, bert_predicted))

0.66
